In [10]:
from matplotlib.pyplot import *
%matplotlib qt

In [11]:
codes = {'pz_x':295004869,
        'pz_y':29500501,
        'pz_z':29500495,
        'sg_x': 84868288,
        'sg_y': 84868295,
        'qpd': 69250571}

In [12]:
codes['pz_x']

295004869

In [13]:
import clr
import sys
import time

#this appends to the global path
sys.path.append("C:\Program Files\Thorlabs\Kinesis")

#This loads the necessary libraries CNET
clr.AddReference("Thorlabs.MotionControl.Benchtop.PiezoCLI")
clr.AddReference("Thorlabs.MotionControl.DeviceManagerCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.PiezoCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.PositionAlignerCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.StrainGaugeCLI")
clr.AddReference("System")

from Thorlabs.MotionControl.KCube.PiezoCLI import KCubePiezo
from Thorlabs.MotionControl.KCube.StrainGaugeCLI import KCubeStrainGauge
from Thorlabs.MotionControl.DeviceManagerCLI import DeviceManagerCLI
from Thorlabs.MotionControl.GenericPiezoCLI.Piezo import *
from Thorlabs.MotionControl.KCube.PositionAlignerCLI import KCubePositionAligner


from System import Decimal

def list_devices():
    """Return a list of Kinesis serial numbers"""
    DeviceManagerCLI.BuildDeviceList()
    return DeviceManagerCLI.GetDeviceList()


In [14]:
list_devices()

['29500489', '29500495', '29500501', '69250571', '84868288', '84868295']

In [12]:
class KPZ():
    
    """
    Builds a K-Cube Piezo device instance for control
    """
    
    def __init__(self, serial_number):
        #serial number
        self.serial_number = str(serial_number)
        
        #builds the list of devices
        DeviceManagerCLI.BuildDeviceList()
        
        #creates a device
        self.device = KCubePiezo.CreateKCubePiezo(self.serial_number)
        
        #connection status
        self.connected = False

    def connect(self):
        #initialize communication
        
        #connect if not connected
        assert not self.connected
        self.device.Connect(self.serial_number)
        self.connected = True
        
        self.device.WaitForSettingsInitialized(5000)
        self.device.StartPolling(250)
        time.sleep(0.5)
        self.device.EnableDevice()
        time.sleep(0.5)
        
        config = self.device.GetPiezoConfiguration(self.serial_number)
        info = self.device.GetDeviceInfo()
        maxVolts = self.device.GetMaxOutputVoltage()
        self.max_v =  Decimal.ToDouble(maxVolts)
        
        return config, info, self.max_v
        
        
    
    def disable(self):
        self.device.DisableDevice()
        
    def close(self):
        
        if not self.connected:
            print(f"Not closing piezo device {self.serial_number}, it's not open!")
            return
        
        self.device.StopPolling()
        self.device.Disconnect(True)

    def set_output_voltages(self, voltage):
        
        if voltage > 0 and voltage<self.max_v:
            #print(self.max_v)
            print(Decimal(voltage))
            i = self.device.SetOutputVoltage(Decimal(voltage))
        else:
            print('Invalid Voltage')
            
    def get_output_voltages(self):
        """Retrieve the output voltages as a list of floating-point numbers"""
        return Decimal.ToDouble(self.device.GetOutputVoltage())

In [4]:
list_devices()

['29500489', '29500495', '29500501', '69250571', '84868288', '84868295']

In [9]:
pz = KPZ(list_devices()[2])
print(pz.connect())

(<Thorlabs.MotionControl.KCube.PiezoCLI.KCubePiezoConfiguration object at 0x0000021A24DA2940>, <Thorlabs.MotionControl.DeviceManagerCLI.SingleDeviceInfo object at 0x0000021A24DA2CD0>, 75.0)


In [15]:
print(pz.get_output_voltages())
pz.set_output_voltages(0.5)

0.0
0.5


In [16]:
print(pz.get_output_voltages())

0.487533188879055


In [23]:
pz.close()

DeviceNotReadyException: Device is not connected
   at Thorlabs.MotionControl.DeviceManagerCLI.ThorlabsGenericCoreDeviceCLI.VerifyDeviceConnected(Int32 functionDepth)
   at Thorlabs.MotionControl.GenericPiezoCLI.Piezo.ThorlabsGenericPiezoCLI.StopPolling()

In [4]:
clr.AddReference("Thorlabs.MotionControl.KCube.PiezoCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.PositionAlignerCLI")
clr.AddReference("Thorlabs.MotionControl.TCube.StrainGaugeCLI")
clr.AddReference("Thorlabs.MotionControl.GenericPiezoCLI")
clr.AddReference("Thorlabs.MotionControl.DeviceManagerCLI")
clr.AddReference("System")

from Thorlabs.MotionControl.KCube.PiezoCLI import KCubePiezo
from Thorlabs.MotionControl.TCube.StrainGaugeCLI import TCubeStrainGauge
from Thorlabs.MotionControl.KCube.PiezoCLI import KCubePiezo
from Thorlabs.MotionControl.DeviceManagerCLI import DeviceManagerCLI
from Thorlabs.MotionControl.GenericPiezoCLI.Piezo import *
from Thorlabs.MotionControl.GenericPiezoCLI.StrainGauge import *
from Thorlabs.MotionControl.GenericPiezoCLI.Settings import *
from Thorlabs.MotionControl.KCube.PositionAlignerCLI import KCubePositionAligner
from Thorlabs.MotionControl.KCube.PositionAlignerCLI import * 

class KPA():
    def __init__(self, serial_number):
        #initializes from a serial number
        self.serial_number = str(serial_number)
        DeviceManagerCLI.BuildDeviceList()
        
        #initializes the device
        self.device = KCubePositionAligner.CreateKCubePositionAligner(self.serial_number)
        
        #initializes the connected state
        self.connected = False
        
        
    def connect(self):
        #initialize communication
        
        #connect if not connected
        assert not self.connected
        self.device.Connect(self.serial_number)
        self.connected = True
        
        self.device.WaitForSettingsInitialized(5000)
        self.device.StartPolling(250)
        time.sleep(0.5)
        self.device.EnableDevice()
        time.sleep(0.5)
        
        self.device.SetOperatingMode(PositionAlignerStatus.OperatingModes.Monitor, False)
        
        info = self.device.GetDeviceInfo()
        
        print(info)
        
        
        
    def close(self):
        
        if not self.connected:
            print(f"Not closing KPA device {self.serial_number}, it's not open!")
            return
        
        self.device.StopPolling()
        self.device.Disconnect(True)
        
    def get_position(self):
        status = self.device.Status
        time.sleep(0.25)
        read_x,read_y = status.PositionDifference.X, self.device.Status.PositionDifference.Y
        read_sum = status.Sum
        return read_x,read_y,read_sum

In [11]:
list_devices()

['29500489', '29500495', '29500501', '69250571', '84868288', '84868295']

In [12]:
kpa = KPA(list_devices()[3])
kpa.connect()
print(kpa.get_position())

Thorlabs.MotionControl.DeviceManagerCLI.SingleDeviceInfo
(-0.0003051850947599719, 0.0003051850947599719, 0.0)


In [11]:
kpa.close()

Not closing KPA device 69250571, it's not open!


In [13]:
clr.AddReference("Thorlabs.MotionControl.KCube.PiezoCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.PositionAlignerCLI")
clr.AddReference("Thorlabs.MotionControl.TCube.StrainGaugeCLI")
clr.AddReference("Thorlabs.MotionControl.GenericPiezoCLI")
clr.AddReference("Thorlabs.MotionControl.DeviceManagerCLI")
clr.AddReference("System")

from Thorlabs.MotionControl.KCube.PiezoCLI import KCubePiezo
from Thorlabs.MotionControl.TCube.StrainGaugeCLI import TCubeStrainGauge
from Thorlabs.MotionControl.KCube.PiezoCLI import KCubePiezo
from Thorlabs.MotionControl.DeviceManagerCLI import DeviceManagerCLI
from Thorlabs.MotionControl.GenericPiezoCLI.Piezo import *
from Thorlabs.MotionControl.GenericPiezoCLI.StrainGauge import *
from Thorlabs.MotionControl.GenericPiezoCLI.Settings import *
from Thorlabs.MotionControl.KCube.PositionAlignerCLI import KCubePositionAligner

class TSG():
    def __init__(self, serial_number):
        #initializes from a serial number
        self.serial_number = str(serial_number)
        DeviceManagerCLI.BuildDeviceList()
        
        #initializes the device
        self.device = TCubeStrainGauge.CreateTCubeStrainGauge(self.serial_number)
        
        #initializes the connected state
        self.connected = False
        
        
    def connect(self):
        #initialize communication
        
        #connect if not connected
        assert not self.connected
        self.device.Connect(self.serial_number)
        self.connected = True
        
        self.device.WaitForSettingsInitialized(5000)
        self.device.StartPolling(250)
        time.sleep(0.5)
        self.device.EnableDevice()
        time.sleep(0.5)
        
        #config = self.device.GetPiezoConfiguration(self.serial_number)
        info = self.device.GetDeviceInfo()
        #maxVolts = self.device.GetMaxOutputVoltage()
        #self.max_v =  Decimal.ToDouble(maxVolts)
        
        print(info)
        
        
        
    def close(self):
        
        if not self.connected:
            print(f"Not closing KSG device {self.serial_number}, it's not open!")
            return
        
        self.device.StopPolling()
        self.device.Disconnect(True)
        

    def get_voltage(self):
        self.device.SetDisplayMode(DisplayModeSettings.TSGDisplayModes.Voltage)
        time.sleep(0.25)
        read = self.device.Status.Reading
        return Decimal.ToDouble(read)
    
    def get_position(self):
        self.device.SetDisplayMode(DisplayModeSettings.TSGDisplayModes.Position)
        time.sleep(0.25)
        read = self.device.Status.Reading
        return Decimal.ToDouble(read)
    
    def zero_position(self):
        self.device.SetDisplayMode(DisplayModeSettings.TSGDisplayModes.Position)
        time.sleep(0.25)
        self.device.SetZero()
        print("zeroing")
        time.sleep(10)
        
    
    def get_max_travel(self):
        return Decimal.ToDouble(tsg.device.GetMaxTravel())

In [6]:
list_devices()

['29500489', '29500495', '29500501', '69250571', '84868288', '84868295']

In [14]:
tsg = TSG(list_devices()[4])
tsg.connect()

NullReferenceException: Object reference not set to an instance of an object.
   at Thorlabs.MotionControl.PrivateInternal.Settings.DeviceConfigurationBase.VerifyDeviceSettingsName(Boolean replace)
   at Thorlabs.MotionControl.PrivateInternal.Settings.DeviceConfigurationBase.ReadConfiguration(DeviceConfigurationNode node, GeneralSettings generalSettings)
   at Thorlabs.MotionControl.PrivateInternal.Settings.DeviceConfigurationBase.LoadConfiguration(String serialNo, GeneralSettings generalSettings)
   at Thorlabs.MotionControl.TCube.StrainGaugeCLI.TCubeStrainGauge.InitializeDevice(String serialNo)
   at Thorlabs.MotionControl.TCube.StrainGaugeCLI.TCubeStrainGauge.CreateTCubeStrainGauge(String serialNo)

In [34]:
print(tsg.get_voltage())
print(tsg.get_position())

-0.0396740623187963
-0.0799584948271126


In [35]:
Decimal.ToDouble(tsg.device.GetMaxTravel())

20.0

In [36]:
tsg.device.SetZero()

In [37]:
tsg.close()

# Testing

In [12]:
import clr 
import sys
import time

#this appends to the global path
sys.path.append("C:\Program Files\Thorlabs\Kinesis")

#This loads the necessary libraries CNET
clr.AddReference("Thorlabs.MotionControl.Benchtop.PiezoCLI")
clr.AddReference("Thorlabs.MotionControl.DeviceManagerCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.PiezoCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.PositionAlignerCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.StrainGaugeCLI")
clr.AddReference("System")

from Thorlabs.MotionControl.KCube.PiezoCLI import KCubePiezo
from Thorlabs.MotionControl.KCube.StrainGaugeCLI import KCubeStrainGauge
from Thorlabs.MotionControl.DeviceManagerCLI import DeviceManagerCLI
from Thorlabs.MotionControl.GenericPiezoCLI.Piezo import *
from Thorlabs.MotionControl.KCube.PositionAlignerCLI import KCubePositionAligner


from System import Decimal

def list_devices():
    """Return a list of Kinesis serial numbers"""
    DeviceManagerCLI.BuildDeviceList()
    return DeviceManagerCLI.GetDeviceList()

In [1]:
clr.AddReference("Thorlabs.MotionControl.KCube.PiezoCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.PositionAlignerCLI")
clr.AddReference("Thorlabs.MotionControl.TCube.StrainGaugeCLI")
clr.AddReference("Thorlabs.MotionControl.GenericPiezoCLI")
clr.AddReference("Thorlabs.MotionControl.DeviceManagerCLI")
clr.AddReference("System")

from Thorlabs.MotionControl.KCube.PiezoCLI import KCubePiezo
from Thorlabs.MotionControl.TCube.StrainGaugeCLI import TCubeStrainGauge
from Thorlabs.MotionControl.KCube.PiezoCLI import KCubePiezo
from Thorlabs.MotionControl.DeviceManagerCLI import DeviceManagerCLI
from Thorlabs.MotionControl.GenericPiezoCLI.Piezo import *
from Thorlabs.MotionControl.GenericPiezoCLI.StrainGauge import *
from Thorlabs.MotionControl.GenericPiezoCLI.Settings import *
from Thorlabs.MotionControl.KCube.PositionAlignerCLI import KCubePositionAligner
from Thorlabs.MotionControl.KCube.PositionAlignerCLI import * 

class KPA():
    def __init__(self, serial_number):
        #initializes from a serial number
        self.serial_number = str(serial_number)
        DeviceManagerCLI.BuildDeviceList()
        
        #initializes the device
        self.device = KCubePositionAligner.CreateKCubePositionAligner(self.serial_number)
        
        #initializes the connected state
        self.connected = False
        
        
    def connect(self):
        #initialize communication
        
        #connect if not connected
        assert not self.connected
        self.device.Connect(self.serial_number)
        self.connected = True
        
        self.device.WaitForSettingsInitialized(5000)
        self.device.StartPolling(250)
        time.sleep(0.5)
        self.device.EnableDevice()
        time.sleep(0.5)
        
        self.device.SetOperatingMode(PositionAlignerStatus.OperatingModes.OpenLoop, False)
        
        info = self.device.GetDeviceInfo()
        
        print(info)
        

    def close(self):
        
        if not self.connected:
            print(f"Not closing KPA device {self.serial_number}, it's not open!")
            return
        
        self.device.StopPolling()
        self.device.Disconnect(True)
        
    def get_position(self):
        status = self.device.Status
        time.sleep(0.25)
        read_x,read_y = status.PositionDifference.X, self.device.Status.PositionDifference.Y
        read_sum = status.Sum
        return read_x,read_y,read_sum

NameError: name 'clr' is not defined

In [14]:
kpa = KPA(list_devices()[3])
kpa.connect()


Thorlabs.MotionControl.DeviceManagerCLI.SingleDeviceInfo


In [15]:
kpa.get_position()

(-0.0003051850947599719, 0.0, 0.0)

In [11]:
kpa.close()

In [1]:
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from pyqtgraph import PlotWidget, plot
import pyqtgraph as pg
import sys  # We need sys so that we can pass argv to QApplication
import os
import numpy as np

class cube_widget(QWidget):

    def __init__(self, parent=None):
        super().__init__()
        
        self.global_layout = QHBoxLayout(self)
        
        self.graphWidget = pg.PlotWidget()
        
        self.global_layout.addWidget(self.graphWidget)

        self.x = [ kpa.get_position()[0] ] 
        self.y = [ kpa.get_position()[1] ]
        
        print(self.x,self.y)
        
        self.graphWidget.setBackground('w')
        
        self.graphWidget.showGrid(x = True, y = True, alpha = 1.0)
        
        self.graphWidget.setXRange(-0.008, 0.008, padding=0)
        self.graphWidget.setYRange(-0.008, 0.008, padding=0)

        pen = pg.mkPen(color=(255, 0, 0))
        self.data_line =  self.graphWidget.plot(self.x, self.y, pen=pen, symbol='o')
        
        self.timer = QTimer()
        self.timer.setInterval(80)
        self.timer.timeout.connect(self.update_plot_data)
        self.timer.start()

    def update_plot_data(self):
        
        print(self.x,self.y)
        
        self.x = [ kpa.get_position()[0] ] 

        self.y = [ kpa.get_position()[1] ]   

        self.data_line.setData(self.x, self.y)  # Update the data.
    
    def closeEvent(self,event):
        self.kpa.close()
        print('It´s closed, bby')
        event.accept()

qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
    #file = QFile(":/dark/stylesheet.qss")
    #file.open(QFile.ReadOnly | QFile.Text)
    #stream = QTextStream(file)
    #qapp.setStyleSheet(stream.readAll())
    
if __name__ == "__main__": 
    #start the widget
    ui = cube_widget()
    #show the widget
    ui.show()
    #start the events loop
    qapp.exec_()

NameError: name 'kpa' is not defined

# Impulse modelling

In [6]:
import clr
import sys
import time

#this appends to the global path
sys.path.append("C:\Program Files\Thorlabs\Kinesis")

#This loads the necessary libraries CNET
clr.AddReference("Thorlabs.MotionControl.Benchtop.PiezoCLI")
clr.AddReference("Thorlabs.MotionControl.DeviceManagerCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.PiezoCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.PositionAlignerCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.StrainGaugeCLI")
clr.AddReference("System")

from Thorlabs.MotionControl.KCube.PiezoCLI import KCubePiezo
from Thorlabs.MotionControl.KCube.StrainGaugeCLI import KCubeStrainGauge
from Thorlabs.MotionControl.DeviceManagerCLI import DeviceManagerCLI
from Thorlabs.MotionControl.GenericPiezoCLI.Piezo import *
from Thorlabs.MotionControl.KCube.PositionAlignerCLI import KCubePositionAligner


from System import Decimal

def list_devices():
    """Return a list of Kinesis serial numbers"""
    DeviceManagerCLI.BuildDeviceList()
    return DeviceManagerCLI.GetDeviceList()

In [7]:
class KPZ():
    
    """
    Builds a K-Cube Piezo device instance for control
    """
    
    def __init__(self, serial_number):
        #serial number
        self.serial_number = str(serial_number)
        
        #builds the list of devices
        DeviceManagerCLI.BuildDeviceList()
        
        #creates a device
        self.device = KCubePiezo.CreateKCubePiezo(self.serial_number)
        
        #connection status
        self.connected = False

    def connect(self):
        #initialize communication
        
        #connect if not connected
        assert not self.connected
        self.device.Connect(self.serial_number)
        self.connected = True
        
        self.device.WaitForSettingsInitialized(5000)
        self.device.StartPolling(250)
        time.sleep(0.5)
        self.device.EnableDevice()
        time.sleep(0.5)
        
        config = self.device.GetPiezoConfiguration(self.serial_number)
        info = self.device.GetDeviceInfo()
        maxVolts = self.device.GetMaxOutputVoltage()
        self.max_v =  Decimal.ToDouble(maxVolts)
        
        return config, info, self.max_v
        
        
    
    def disable(self):
        self.device.DisableDevice()
        
    def close(self):
        
        if not self.connected:
            print(f"Not closing piezo device {self.serial_number}, it's not open!")
            return
        
        self.device.StopPolling()
        self.device.Disconnect(True)

    def set_output_voltages(self, voltage):
        
        if voltage > 0 and voltage<self.max_v:
            #print(self.max_v)
            print(Decimal(voltage))
            i = self.device.SetOutputVoltage(Decimal(voltage))
        else:
            print('Invalid Voltage')
            
    def get_output_voltages(self):
        """Retrieve the output voltages as a list of floating-point numbers"""
        return Decimal.ToDouble(self.device.GetOutputVoltage())

In [8]:
clr.AddReference("Thorlabs.MotionControl.KCube.PiezoCLI")
clr.AddReference("Thorlabs.MotionControl.KCube.PositionAlignerCLI")
clr.AddReference("Thorlabs.MotionControl.TCube.StrainGaugeCLI")
clr.AddReference("Thorlabs.MotionControl.GenericPiezoCLI")
clr.AddReference("Thorlabs.MotionControl.DeviceManagerCLI")
clr.AddReference("System")

from Thorlabs.MotionControl.KCube.PiezoCLI import KCubePiezo
from Thorlabs.MotionControl.TCube.StrainGaugeCLI import TCubeStrainGauge
from Thorlabs.MotionControl.KCube.PiezoCLI import KCubePiezo
from Thorlabs.MotionControl.DeviceManagerCLI import DeviceManagerCLI
from Thorlabs.MotionControl.GenericPiezoCLI.Piezo import *
from Thorlabs.MotionControl.GenericPiezoCLI.StrainGauge import *
from Thorlabs.MotionControl.GenericPiezoCLI.Settings import *
from Thorlabs.MotionControl.KCube.PositionAlignerCLI import KCubePositionAligner

class TSG():
    def __init__(self, serial_number):
        #initializes from a serial number
        self.serial_number = str(serial_number)
        DeviceManagerCLI.BuildDeviceList()
        
        #initializes the device
        self.device = TCubeStrainGauge.CreateTCubeStrainGauge(self.serial_number)
        
        #initializes the connected state
        self.connected = False
        
        
    def connect(self):
        #initialize communication
        
        #connect if not connected
        assert not self.connected
        self.device.Connect(self.serial_number)
        self.connected = True
        
        self.device.WaitForSettingsInitialized(5000)
        self.device.StartPolling(250)
        time.sleep(0.5)
        self.device.EnableDevice()
        time.sleep(0.5)
        
        #config = self.device.GetPiezoConfiguration(self.serial_number)
        info = self.device.GetDeviceInfo()
        #maxVolts = self.device.GetMaxOutputVoltage()
        #self.max_v =  Decimal.ToDouble(maxVolts)
        
        print(info)
        
        
        
    def close(self):
        
        if not self.connected:
            print(f"Not closing KSG device {self.serial_number}, it's not open!")
            return
        
        self.device.StopPolling()
        self.device.Disconnect(True)
        

    def get_voltage(self):
        self.device.SetDisplayMode(DisplayModeSettings.TSGDisplayModes.Voltage)
        time.sleep(0.25)
        read = self.device.Status.Reading
        return Decimal.ToDouble(read)
    
    def get_position(self):
        self.device.SetDisplayMode(DisplayModeSettings.TSGDisplayModes.Position)
        time.sleep(0.25)
        read = self.device.Status.Reading
        return Decimal.ToDouble(read)
    
    def zero_position(self):
        self.device.SetDisplayMode(DisplayModeSettings.TSGDisplayModes.Position)
        time.sleep(0.25)
        self.device.SetZero()
        print("zeroing")
        time.sleep(10)
        
    
    def get_max_travel(self):
        return Decimal.ToDouble(tsg.device.GetMaxTravel())

In [9]:
#ind: 0 for x_cube, 1 for z_cube, 2 for y_cube
def impulse(ind,v):
    cubes=[KPZ(list_devices()[0]),KPZ(list_devices()[1]),KPZ(list_devices()[2])]
    tsg=TSG(list_devices()[4])
    tsg.connect()
    cubes[ind].connect()
    cubes[ind].set_output_voltages(v)
    time.sleep(0.1)
    a=cubes[ind].get_output_voltages()
    b=tsg.get_position()
    print('Impulse voltage applied: ' + str(a))
    print('Position Measured: ' + str(b))
    x=input('Press Enter to disable...')
    cubes[ind].disable()
    time.sleep(0.2)
    c=cubes[ind].get_output_voltages()
    print('Impulse voltage applied: ' + str(c))
    cubes[ind].close()
    tsg.close()

In [11]:
impulse(2,10) #impulse to x-cube of _ Volts

Thorlabs.MotionControl.DeviceManagerCLI.SingleDeviceInfo
10
Impulse voltage applied: 9.97497482222968
Position Measured: -0.443128757591479
Press Enter to disable...
Impulse voltage applied: 74.9427777947325


### 